최신 체인 구성방법 V0.3
```
LLMChain, SequentialChain 등과 같이 클래스의존도를 줄임
Runnable 공통인터페이스를 통해 일관성을 유지
핵심 : Runnable + Composition  --> 프롬프트 | 모델 | 파서
```

In [1]:
# %pip install langchain openai python-dotenv

In [3]:
from dotenv import load_dotenv
load_dotenv() # .env파일의 내용일 읽어서 환경변수에 등록
import os
openai_key = os.getenv('OPENAI_API_KEY')
print(openai_key[:5]+'****')

sk-pr****


단일체인 : prompt -> llm -> 출력파서 (상품설명)

In [17]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. 프롬프트 탬플릿을 정의
product_prompt = PromptTemplate.from_template(
    "제품 이름 : {product}\n"
    "이 제품의 특징과 장점을 잘 표현할수 있는 한개의 문장으로 만들어 주세요"
)
print(product_prompt.format(product = '선풍기'))
# 2. 출력파서 정의
output_parser = StrOutputParser()

# 3. llm 정의
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)

# 4. LCEL 체인 구성 프롬프트->모델->출력파서
proudct_chain = product_prompt | llm | output_parser  # runnable 객체

result = proudct_chain.invoke({'product' : '선풍기'})

print(result)



제품 이름 : 선풍기
이 제품의 특징과 장점을 잘 표현할수 있는 한개의 문장으로 만들어 주세요
"강력한 바람과 조용한 작동으로 여름철 더위를 시원하게 날려주는 선풍기입니다."


In [ ]:
# OpenAI는 Text completion  문장을 이어서 완성하는 모델...
# text-davinci 모델을 위한 라이브러리 이전방식이라서 .역활기반 템플릿이 없고 단순한 prompt

from langchain_openai import OpenAI
llm = OpenAI(model='gpt-4o-mini',temperature=0.5)
print(llm.invoke('다음 문장을 완성하세요: 옛날옛적에 호랑이가 살았습니다.'))


다중체인 : 체인 합성 및 Runnable 병합(이메일 생성)
```
둘 이상의 llm 호출을 연결해서 복잡한 작업을 수행
"주어진 상황에 대한 이메일 작성" --> 제목
제목을 활용해서 이메일 본문을 작성 --> 본문
chain composition
```


In [18]:
# 프롬프트 1 : 사용자로부터 받은 이메을 요청내용을 입력받아서 "이메일 제목"을 한문장으로 생성하는 명령
# llm 호출 -> 이메일 제목출력('프로젝트 진행상황 회의 일정 안내')
# 중간 출력 변환 : 생성된 제목문자열을 {subject} 키를 갖는 dictionary 변환
# 프롬프트 2 : {subject}변수를 받아서 해당 제목을 가진 이메일 본문내용을 요청
# llm 호출

In [23]:
from langchain_core.runnables import RunnablePassthrough
# 1. 이메일 생성용 프롬프트 정의
subject_prompt = PromptTemplate.from_template(
    '다음 요청 내용을 바탕으로 이메일 제목을 만들어주세요\n'
    '{content}'
)
# 2. 이메일 본문 생성용 프롬프트
body_prompt = PromptTemplate.from_template(
    '위에서 생성된 제목을 활용해서 팀에게 보내는 정중한 이메일 제목과 본문을 작성해 주세요\n'    
    '제목:{subject}\n'
    '본문:'
)
# 3. 두 프롬프트를 결합한 체인 구성
email_chain = (
    subject_prompt
    | llm
    | {'subject' : RunnablePassthrough()}  # 출력된 제목을 subjet 키로 매핑
    | body_prompt
    | llm
    | StrOutputParser()
)
# 4. 실행
result =  email_chain.invoke({
    'content' : '''대선이 1주일 앞으로 다가온 가운데 후보들은 오늘(27일) 마지막 TV 토론에 나섭니다.
                이번엔 '정치'를 주제로 치열한 공방을 벌일 예정입니다.'''
})
print(result)

제목: 대선 1주일 앞두고, 후보들의 마지막 TV 토론: 정치 주제로 치열한 공방!

안녕하세요, 팀 여러분.

다가오는 대선이 일주일 앞으로 다가왔습니다. 이에 따라 후보들의 마지막 TV 토론이 예정되어 있습니다. 이번 토론에서는 각 후보들이 정치적 입장을 더욱 명확히 하고, 유권자들에게 강한 인상을 남기기 위한 치열한 공방이 예상됩니다.

우리 팀도 이 중요한 시점을 놓치지 않고, 후보들의 발언과 정책을 면밀히 분석하여 향후 전략에 반영할 수 있도록 준비해 주시기 바랍니다. 각자의 역할에 맞춰 필요한 자료를 수집하고, 논의할 사항들을 정리해 주시면 감사하겠습니다.

모두의 협력이 중요한 시점이니, 적극적인 참여 부탁드립니다.

감사합니다.

[당신의 이름]  
[당신의 직책]  
[회사/팀 이름]  


조건분기 : 입력조건에 따라 요약 또는 이메일 작성

In [ ]:
from langchain_core.runnables import RunnableBranch, RunnableLambda
# 1. 요약체인(prompt->llm)
summary_prompt = PromptTemplate.from_template(
    '다음 문장을 한 문단으로 간결하게 요약해주세요\n'
    '{text}'
    )
summary_chain = summary_prompt | llm
# 2. email chain은 재활용
# 3. 분기조건 함수 정의 =>Runnable로 래핑
def is_summary_request(user_input: str) -> bool:
    return user_input.strip().startswith('요약:')
condition = RunnableLambda(is_summary_request)

